## Facility Location problem using gurobipy

In [ ]:
import pandas as pd
from gurobipy import *

In [3]:
basic_info = pd.read_excel('IP_dataset.xlsx', 'Basic information')

In [4]:
cities = range(len(basic_info['City']))
markets = range(len(basic_info['Market']))

In [6]:
city_info = pd.read_excel('IP_dataset.xlsx', "City's information")

In [12]:
operating_costs = city_info['Operating cost']
capacities = city_info['Capacity']

In [13]:
market_info = pd.read_excel('IP_dataset.xlsx', "Market's information")

In [14]:
demands = market_info['Demand']

In [17]:
shipping_info = pd.read_excel('IP_dataset.xlsx', "Shipping cost", index_col = 0)

In [22]:
shipping_costs = []

for i in shipping_info.index:
    shipping_costs.append(list(shipping_info.loc[i]))

In [23]:
shipping_costs

[[2.4, 3.25, 4.05, 5.25, 6.95],
 [3.5, 2.3, 3.25, 6.05, 5.85],
 [4.8, 3.4, 2.85, 4.3, 4.8],
 [6.8, 5.25, 4.3, 3.25, 2.1],
 [5.75, 6.0, 4.75, 2.75, 3.5]]

In [29]:
eg2 = Model("eg2")

In [30]:
x = []

for j in cities:
    x.append(eg2.addVar(lb = 0, vtype = GRB.BINARY, name = "x" + str(j+1)))

In [32]:
y = []

for i in markets:
    y.append([])
    
    for j in cities:
        y[i].append(eg2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "y" + str(i+1) + str(j+1)))

In [34]:
#setting the objective function

eg2.setObjective(quicksum(operating_costs[j]*x[j] for j in cities) + quicksum(quicksum(shipping_costs[i][j] * y[i][j] for j in cities) for i in markets), GRB.MINIMIZE)

In [37]:
#setting constraints

eg2.addConstrs((quicksum(y[i][j] for i in markets) <= capacities[j] * x[j] for j in cities), "product_capacity")
eg2.addConstrs((quicksum(y[i][j] for j in cities) >= demands[i] for i in markets), "demand_fulfillment")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [38]:
eg2.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 15 rows, 30 columns and 85 nonzeros
Model fingerprint: 0xfbb419c1
Variable types: 25 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [2e+00, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+03, 2e+04]
Presolve removed 5 rows and 0 columns
Presolve time: 0.12s
Presolved: 10 rows, 30 columns, 55 nonzeros
Variable types: 25 continuous, 5 integer (5 binary)

Root relaxation: objective 2.528000e+05, 11 iterations, 0.06 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 252800.000    0    5          - 252800.000      -     -    0s
H    0     0                    313600.00000 252800.000  19.4%     -    0s
H    0     0                    280400.000

In [50]:
print("Result:")

for j in cities:
    print(x[j].varName, '=', x[j].x)
# head of the result table
print("\tMarket1\tMarket2\tMarket3\tMarket4\tMarket5")

for j in cities:
    # mark which product is printed now
    print("City" + str(j+1), "\t", end="")
    for i in markets:
        # print values of each kind of product
        if len(str(y[i][j].x)) < 7:
            print(y[i][j].x, "\t", end="")
        else:
            print(y[i][j].x, "", end="")
    print("")    # use for change line

print("z* =", eg2.objVal)    # print objective value

Result:
x1 = 0.0
x2 = 1.0
x3 = 0.0
x4 = 1.0
x5 = 1.0
	Market1	Market2	Market3	Market4	Market5
City1 	0.0 	0.0 	0.0 	0.0 	0.0 	
City2 	7999.999999999998 12000.000000000002 0.0 	0.0 	0.0 	
City3 	0.0 	0.0 	0.0 	0.0 	0.0 	
City4 	0.0 	0.0 	8000.000000000004 0.0 	16999.999999999996 
City5 	0.0 	0.0 	999.9999999999964 14000.0 3.637978807091713e-12 
z* = 268950.0
